In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Cài thư viện
!pip install vncorenlp -q

# Tải VnCoreNLP (Chỉ cần cái này, không cần Stopwords nữa)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar -P vncorenlp/ -q
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab -P vncorenlp/models/wordsegmenter/ -q
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr -P vncorenlp/models/wordsegmenter/ -q

print(" Cài đặt hoàn tất!")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 Cài đặt hoàn tất!


In [2]:
from vncorenlp import VnCoreNLP
import pandas as pd
import numpy as np
import re
import os
from tqdm.notebook import tqdm

# Khởi tạo bộ tách từ
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

# Đường dẫn (Đảm bảo đúng folder của bạn)
DRIVE_FOLDER = "/content/drive/My Drive/NLP Project/Data"
INPUT_FILE = os.path.join(DRIVE_FOLDER, "vnexpress_full_dataset.csv")
OUTPUT_FILE = os.path.join(DRIVE_FOLDER, "vnexpress_processed_vncorenlp_for_phobert.csv")

In [3]:
def clean_text_for_bert(text):
    """Làm sạch nhưng GIỮ LẠI CẤU TRÚC CÂU"""
    if not isinstance(text, str): return ""

    # 1. Chuyển thường (Giảm nhiễu cho bài toán phân loại)
    text = text.lower()

    # 2. Xóa HTML, URL, Email
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)

    # 3. Giữ lại Tiếng Việt, Số và Dấu câu cơ bản (.,?!)
    text = re.sub(r'[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ0-9.,?!]', ' ', text)

    # 4. Xóa khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def process_text_pipeline(text):
    """Quy trình: Clean -> Segment (Giữ nguyên Stopwords)"""
    # Bước 1: Clean
    text = clean_text_for_bert(text)
    if not text: return ""

    try:
        # Bước 2: Tách từ bằng VnCoreNLP
        sentences = rdrsegmenter.tokenize(text)

        # Bước 3: Nối lại thành chuỗi
        segmented_text = " ".join([" ".join(sentence) for sentence in sentences])

        return segmented_text
    except:
        return ""

In [4]:
try:
    print(f"\n Đang đọc file từ Drive: '{INPUT_FILE}'...")
    df = pd.read_csv(INPUT_FILE)
    print(f"   - Tổng số bài thô: {len(df)}")

    # 1. Xóa trùng lặp
    df.drop_duplicates(subset=['content'], inplace=True)

    # 2. Xóa bài lỗi/rỗng
    df.dropna(subset=['content'], inplace=True)

    # 3. Chạy xử lý
    print("\n Đang xử lý...")
    tqdm.pandas()
    df['content_processed'] = df['content'].progress_apply(process_text_pipeline)

    # 4. Lọc bài quá ngắn (dưới 50 ký tự thì ko đủ thông tin phân loại)
    df = df[df['content_processed'].str.len() > 50]

    # 5. Mã hóa nhãn (Label Encoding)
    label_mapping = {label: idx for idx, label in enumerate(df['label'].unique())}
    df['label_id'] = df['label'].map(label_mapping)

    print("\n Thống kê dữ liệu sau xử lý:")
    print(df['label'].value_counts())
    print("\n Mapping nhãn:", label_mapping)

    # 6. Lưu file
    df_final = df[['content_processed', 'label', 'label_id']]
    df_final.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

    print(f"\n HOÀN TẤT! File đã lưu tại: {OUTPUT_FILE}")
    print("\n---VÍ DỤ DỮ LIỆU CHUẨN---")
    print(df_final['content_processed'].iloc[0][:300])

except FileNotFoundError:
    print(f" Lỗi: Không tìm thấy file '{INPUT_FILE}'. Kiểm tra lại đường dẫn Drive.")


 Đang đọc file từ Drive: '/content/drive/My Drive/NLP Project/Data/vnexpress_full_dataset.csv'...
   - Tổng số bài thô: 13928

 Đang xử lý...


  0%|          | 0/13730 [00:00<?, ?it/s]


 Thống kê dữ liệu sau xử lý:
label
Kinh_doanh            1491
The_gioi              1489
Doi_song              1485
Khoa_hoc_cong_nghe    1472
The_thao              1467
Suc_khoe              1444
Giao_duc              1406
Thu_gian              1224
Xe                    1148
Phap_luat             1104
Name: count, dtype: int64

 Mapping nhãn: {'The_thao': 0, 'Suc_khoe': 1, 'Giao_duc': 2, 'Phap_luat': 3, 'Kinh_doanh': 4, 'Thu_gian': 5, 'Khoa_hoc_cong_nghe': 6, 'Xe': 7, 'Doi_song': 8, 'The_gioi': 9}

 HOÀN TẤT! File đã lưu tại: /content/drive/My Drive/NLP Project/Data/vnexpress_processed_vncorenlp_for_phobert.csv

---VÍ DỤ DỮ LIỆU CHUẨN---
brazilthủ môn gremio anapolis ramon souza bị bắn thủng đùi sau trận thua centro oestes 1 2 trên sân_nhà ở giải hạng nhì tiểu_bang goias tối 10 7 . cảnh_sát brazil bắn cầu_thủ trên sân tan trận , một nhóm cầu_thủ gremio lao vào tấn_công trọng_tài rồi ẩu_đả với cầu_thủ đối_phương centro oestes . cảnh_
